# Imports + Drive + paths

In [2]:
!pip install mediapipe==0.10.14 opencv-python==4.10.0.84 scikit-learn tensorflowjs --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 77.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.2.3 requires packaging>=23.0, but you have packaging 20.9 which is incompatible.
db-dtypes 1.4.4 requires packaging>=24.2.0, but you have packaging 20.9 which is incompatible.
shap 0.50.0 requires packaging>20.9, but you

In [19]:
# 0. IMPORTS BÁSICOS
from google.colab import drive
drive.mount('/content/drive')

import os
import cv2
import json
import numpy as np
from collections import defaultdict

import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("TF version:", tf.__version__)
print("OpenCV version:", cv2.__version__)

# 0.1 RUTAS DEL PROYECTO
BASE_DIR = "/content/drive/MyDrive/SignProject"
RAW_DIR = os.path.join(BASE_DIR, "raw_videos")   # antes decía "raw"
PROCESSED_DIR = os.path.join(BASE_DIR, "processed")
MODELS_DIR = os.path.join(BASE_DIR, "models")

os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

print("RAW_DIR:", RAW_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)
print("MODELS_DIR:", MODELS_DIR)


mp_holistic = mp.solutions.holistic


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
TF version: 2.19.0
OpenCV version: 4.12.0
RAW_DIR: /content/drive/MyDrive/SignProject/raw_videos
PROCESSED_DIR: /content/drive/MyDrive/SignProject/processed
MODELS_DIR: /content/drive/MyDrive/SignProject/models


1. Función de extracción de landmarks (misma lógica que entrenamiento)

In [20]:
def extract_landmarks_from_results(results):
    """
    Convierte los resultados de MediaPipe Holistic en un vector 1D (225,)
    con pose (33), mano izq (21) y mano der (21).
    Cada punto = (x, y, z) => 75 * 3 = 225 features.
    """
    def get_xyz(landmarks, n_points):
        if landmarks is None:
            data = [[0.0, 0.0, 0.0]] * n_points
        else:
            data = [[lm.x, lm.y, lm.z] for lm in landmarks]
            if len(data) < n_points:
                # Rellenar si faltan
                data += [[0.0, 0.0, 0.0]] * (n_points - len(data))
            data = data[:n_points]
        return data

    pose = get_xyz(results.pose_landmarks.landmark if results.pose_landmarks else None, 33)
    left_hand = get_xyz(results.left_hand_landmarks.landmark if results.left_hand_landmarks else None, 21)
    right_hand = get_xyz(results.right_hand_landmarks.landmark if results.right_hand_landmarks else None, 21)

    all_points = pose + left_hand + right_hand
    return np.array(all_points, dtype=np.float32).flatten()  # (225,)


def mirror_sequence(seq):
    """
    Recibe una secuencia (T, 225) y devuelve una versión espejada:
      - Refleja en horizontal: x -> 1.0 - x
      - Intercambia mano izquierda y mano derecha

    Asume el orden:
      0-32  -> pose (33 puntos)
      33-53 -> mano izquierda (21 puntos)
      54-74 -> mano derecha (21 puntos)
    """
    if seq.ndim != 2 or seq.shape[1] != 225:
        raise ValueError(f"mirror_sequence espera (T, 225), recibió {seq.shape}")

    T, D = seq.shape
    pts = seq.reshape(T, 75, 3).copy()  # (T, 75, 3)

    # 1) Espejar en eje X (horizontal)
    pts[..., 0] = 1.0 - pts[..., 0]

    # 2) Separar segmentos: pose, mano izq, mano der
    pose   = pts[:, :33, :]      # (T, 33, 3)
    left   = pts[:, 33:54, :]    # (T, 21, 3)
    right  = pts[:, 54:75, :]    # (T, 21, 3)

    # 3) Reconstruir como: pose + (mano derecha como izquierda) + (mano izquierda como derecha)
    pts_mirrored = np.concatenate([pose, right, left], axis=1)  # (T, 75, 3)

    # 4) Volver a (T, 225)
    return pts_mirrored.reshape(T, 225).astype(np.float32)



**2. Construcción del dataset secuencial para LSTM**

Cada video = 1 muestra
Cada muestra = secuencia de max_frames frames, cada frame = 225 features.

In [21]:
def build_sequence_dataset_from_raw(
    RAW_DIR,
    outfile="landmarks_sequences_v1.npz",
    max_frames=20
):
    """
    Crea un dataset secuencial:
      X_seq -> (n_videos, max_frames, 225)
      y_seq -> (n_videos,)
    Cada video = 1 muestra (secuencia de hasta max_frames).
    Si el video tiene menos frames, se rellena con ceros (padding).
    """
    classes = sorted(os.listdir(RAW_DIR))
    classes = [c for c in classes if os.path.isdir(os.path.join(RAW_DIR, c))]
    print("Clases encontradas:", classes)

    label_map = {cls_name: i for i, cls_name in enumerate(classes)}
    print("label_map:", label_map)

    X_sequences = []
    y_sequences = []
    label_names = classes  # para guardar luego

    with mp_holistic.Holistic(
        static_image_mode=False,
        model_complexity=1,
        enable_segmentation=False,
        refine_face_landmarks=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as holistic:

        for cls_name in classes:
            class_dir = os.path.join(RAW_DIR, cls_name)
            videos = [f for f in os.listdir(class_dir)
                      if f.lower().endswith((".mp4", ".mov", ".avi", ".mkv"))]

            print(f"\nProcesando clase '{cls_name}' ({len(videos)} videos)...")

            for vid_name in videos:
                video_path = os.path.join(class_dir, vid_name)
                cap = cv2.VideoCapture(video_path)

                frames_feats = []

                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break

                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    results = holistic.process(frame_rgb)

                    vec = extract_landmarks_from_results(results)  # (225,)
                    frames_feats.append(vec)

                    # Si ya tenemos max_frames, paramos
                    if len(frames_feats) >= max_frames:
                        break

                cap.release()

                if len(frames_feats) == 0:
                    print(f"⚠️ Video sin frames válidos: {video_path}")
                    continue



                                # Convertir a array (n_frames, 225)
                seq = np.array(frames_feats, dtype=np.float32)

                # Pad o recortar a max_frames
                if seq.shape[0] < max_frames:
                    pad_len = max_frames - seq.shape[0]
                    pad = np.zeros((pad_len, seq.shape[1]), dtype=np.float32)
                    seq = np.concatenate([seq, pad], axis=0)
                else:
                    seq = seq[:max_frames, :]

                # ================================
                #  👇  AQUI DUPLICAMOS CON ESPEJO
                # ================================

                # Secuencia original
                X_sequences.append(seq)               # (max_frames, 225)
                y_sequences.append(label_map[cls_name])

                # Secuencia espejada (cambia mano dominante)
                try:
                    seq_mirror = mirror_sequence(seq)     # (max_frames, 225)
                    X_sequences.append(seq_mirror)
                    y_sequences.append(label_map[cls_name])
                except Exception as e:
                    print(f"⚠️ Error espejando video {video_path}: {e}")








    X_seq = np.stack(X_sequences, axis=0)      # (n_videos, max_frames, 225)
    y_seq = np.array(y_sequences, dtype=np.int32)

    print("\n=== RESUMEN SECUENCIAL ===")
    print("X_seq shape:", X_seq.shape)  # (n_videos, max_frames, 225)
    print("y_seq shape:", y_seq.shape)

    out_path = os.path.join(PROCESSED_DIR, outfile)
    np.savez(out_path, X_seq=X_seq, y_seq=y_seq, label_names=np.array(label_names))
    print("Guardado en:", out_path)

    return X_seq, y_seq, label_names


# EJECUTAR UNA VEZ PARA CREAR DATASET
MAX_FRAMES = 20  # puedes cambiarlo, pero mantén el mismo valor en todo el notebook

X_seq, y_seq, label_names = build_sequence_dataset_from_raw(
    RAW_DIR,
    outfile="landmarks_sequences_v1.npz",
    max_frames=MAX_FRAMES
)


Clases encontradas: ['Azul', 'Crema', 'Marron', 'Negro', 'Verde']
label_map: {'Azul': 0, 'Crema': 1, 'Marron': 2, 'Negro': 3, 'Verde': 4}

Procesando clase 'Azul' (3 videos)...

Procesando clase 'Crema' (3 videos)...

Procesando clase 'Marron' (3 videos)...

Procesando clase 'Negro' (3 videos)...

Procesando clase 'Verde' (3 videos)...

=== RESUMEN SECUENCIAL ===
X_seq shape: (30, 20, 225)
y_seq shape: (30,)
Guardado en: /content/drive/MyDrive/SignProject/processed/landmarks_sequences_v1.npz


3. Carga de dataset secuencial + split por porcentaje

In [23]:
# 3. CARGA DE DATASET SECUENCIAL Y SPLIT ESTRATIFICADO

from sklearn.model_selection import train_test_split
import numpy as np
import os

data_path = os.path.join(PROCESSED_DIR, "landmarks_sequences_v1.npz")
data = np.load(data_path, allow_pickle=True)

# OJO: las claves que guardamos en la parte 2 son estas:
#   X_seq -> (n_videos, max_frames, 225)
#   y_seq -> (n_videos,)
#   label_names -> lista de nombres de clase
X_seq = data["X_seq"]
y_seq = data["y_seq"]              # ya son enteros 0,1,2,...
label_names = data["label_names"].tolist()

print("X_seq:", X_seq.shape)
print("y_seq:", y_seq.shape)
print("Clases:", label_names)

# ---- Split por porcentaje pero robusto ----
n_samples = X_seq.shape[0]
classes_unique = np.unique(y_seq)
n_classes = len(classes_unique)

val_ratio = 0.2  # ~20% validación

# n_val aproximado
n_val = int(round(n_samples * val_ratio))

# asegurar al menos 1 muestra por clase en VALIDACIÓN
n_val = max(n_val, n_classes)

# asegurar al menos 1 muestra por clase en TRAIN
n_val = min(n_val, n_samples - n_classes)

print(f"Total muestras: {n_samples}, clases: {n_classes}, val: {n_val}")

X_train_seq, X_val_seq, y_train_seq, y_val_seq = train_test_split(
    X_seq,
    y_seq,
    test_size=n_val,   # entero: sklearn usa exactamente ese número
    stratify=y_seq,
    random_state=42,
)

print("Train seq:", X_train_seq.shape, y_train_seq.shape)
print("Val seq:",   X_val_seq.shape,   y_val_seq.shape)


X_seq: (30, 20, 225)
y_seq: (30,)
Clases: ['Azul', 'Crema', 'Marron', 'Negro', 'Verde']
Total muestras: 30, clases: 5, val: 6
Train seq: (24, 20, 225) (24,)
Val seq: (6, 20, 225) (6,)


4. Modelo LSTM (temporal)

In [24]:
# 4. MODELO LSTM TEMPORAL

from tensorflow import keras
from tensorflow.keras import layers

num_classes = len(label_names)
max_frames = X_seq.shape[1]
n_features = X_seq.shape[2]

model_lstm = keras.Sequential([
    layers.Input(shape=(max_frames, n_features)),   # (T, 225)

    # Ignora frames de padding (todo ceros)
    layers.Masking(mask_value=0.0),

    layers.LSTM(128, return_sequences=False),
    layers.Dropout(0.3),

    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),

    layers.Dense(num_classes, activation='softmax')
])

model_lstm.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model_lstm.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_1 (Masking)             │ (None, 20, 225)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       181,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 189,829 (741.52 KB)

 Trainable params: 189,829 (741.52 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
# 4.1 ENTRENAMIENTO

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20,
    restore_best_weights=True
)

history_lstm = model_lstm.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=450,
    batch_size=4,
    callbacks=[early_stopping],
    verbose=1
)

val_loss, val_acc = model_lstm.evaluate(X_val_seq, y_val_seq, verbose=0)
print("\nVAL ACCURACY LSTM:", val_acc)


Epoch 1/450
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.5089 - loss: 1.4660 - val_accuracy: 0.0000e+00 - val_loss: 1.7954
Epoch 2/450
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.3893 - loss: 1.4056 - val_accuracy: 0.0000e+00 - val_loss: 1.8332
Epoch 3/450
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.2244 - loss: 1.5628 - val_accuracy: 0.1667 - val_loss: 1.8641
Epoch 4/450
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3327 - loss: 1.4099 - val_accuracy: 0.1667 - val_loss: 1.8512
Epoch 5/450
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5190 - loss: 1.2626 - val_accuracy: 0.3333 - val_loss: 1.7743
Epoch 6/450
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5137 - loss: 1.2635 - val_accuracy: 0.3333 - val_loss: 1.7850
Epoch 7/450
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6673 - loss: 1.2457 - val_accuracy: 0.3333 - val_loss: 1.8105
Epoch 8/450
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6958 - loss: 1.0576 - val_accuracy: 0.3333 - 

In [49]:
keras_path_lstm = os.path.join(MODELS_DIR, "sign_model_lstm_v1.keras")
model_lstm.save(keras_path_lstm)
print("Modelo LSTM guardado en:", keras_path_lstm)

labels_path = os.path.join(MODELS_DIR, "label_names.json")

with open(labels_path, "w") as f:
    json.dump(label_names, f)

print("Listo. Guardado en:", labels_path)
print(label_names)



Modelo LSTM guardado en: /content/drive/MyDrive/SignProject/models/sign_model_lstm_v1.keras
Listo. Guardado en: /content/drive/MyDrive/SignProject/models/label_names.json
['Azul', 'Crema', 'Marron', 'Negro', 'Verde']


In [50]:
loss_full, acc_full = model_lstm.evaluate(X_seq, y_seq, verbose=0)
print(f"Accuracy sobre TODO el dataset (solo referencia): {acc_full:.2f}")


Accuracy sobre TODO el dataset (solo referencia): 0.63


**6. Función para probar un video cualquiera (LSTM)**

In [51]:
# 6. USO DEL MODELO LSTM CON UN VIDEO



def load_lstm_model_and_labels():
    model_path = os.path.join(MODELS_DIR, "sign_model_lstm_v1.keras")
    labels_path = os.path.join(MODELS_DIR, "label_names.json")

    model = keras.models.load_model(model_path)

    with open(labels_path, "r") as f:
        label_names = json.load(f)

    return model, label_names



def preprocess_video_to_sequence(video_path, max_frames=20):
    """
    Procesa un video con MediaPipe Holistic y devuelve una secuencia
    (1, max_frames, 225) lista para el modelo LSTM.
    """
    cap = cv2.VideoCapture(video_path)

    frames_feats = []
    with mp_holistic.Holistic(
        static_image_mode=False,
        model_complexity=1,
        enable_segmentation=False,
        refine_face_landmarks=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as holistic:

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(frame_rgb)

            vec = extract_landmarks_from_results(results)  # (225,)
            frames_feats.append(vec)

            if len(frames_feats) >= max_frames:
                break

    cap.release()

    if len(frames_feats) == 0:
        raise ValueError(f"Video sin frames válidos: {video_path}")

    seq = np.array(frames_feats, dtype=np.float32)

    # Pad o recortar
    if seq.shape[0] < max_frames:
        pad_len = max_frames - seq.shape[0]
        pad = np.zeros((pad_len, seq.shape[1]), dtype=np.float32)
        seq = np.concatenate([seq, pad], axis=0)
    else:
        seq = seq[:max_frames, :]

    seq = seq.reshape(1, max_frames, seq.shape[1])  # (1, T, 225)
    return seq


def predict_video_lstm(video_path, max_frames=20):
    model, label_names = load_lstm_model_and_labels()
    seq = preprocess_video_to_sequence(video_path, max_frames=max_frames)

    probs = model.predict(seq, verbose=0)[0]  # (num_classes,)
    idx = int(np.argmax(probs))
    label = label_names[idx]
    conf = float(probs[idx])

    print(f"\nVideo: {video_path}")
    print(f"Predicción LSTM: {label} (confianza {conf:.2f})")
    print("\nDistribución de probabilidades:")
    for i, name in enumerate(label_names):
        print(f"  {name}: {probs[i]:.2f}")

    return label, conf, probs


In [52]:
test_video_path = os.path.join(BASE_DIR, "raw_videos", "Marron", "Marronf.mp4")
predict_video_lstm(test_video_path, max_frames=MAX_FRAMES)



Video: /content/drive/MyDrive/SignProject/raw_videos/Marron/Marronf.mp4
Predicción LSTM: Marron (confianza 0.48)

Distribución de probabilidades:
  Azul: 0.07
  Crema: 0.30
  Marron: 0.48
  Negro: 0.07
  Verde: 0.08


('Marron',
 0.48444125056266785,
 array([0.06586322, 0.29921758, 0.48444125, 0.06920952, 0.08126838],
       dtype=float32))